In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df = pd.read_csv('data/movie_metadata.csv')

In [11]:
for column in df.select_dtypes(['float64','int64']).columns:
    df[column] = df[column].fillna(value=np.mean(df[column]))

In [12]:
df.dropna(inplace=True)

In [13]:
df.drop(columns=['aspect_ratio','title_year','facenumber_in_poster','director_facebook_likes','actor_3_facebook_likes','actor_1_facebook_likes','gross','cast_total_facebook_likes','actor_2_facebook_likes', 'budget'],inplace=True)

In [14]:
for column in ['genres','color','content_rating']:
    dummies = pd.DataFrame(pd.Series(df[column]).str.get_dummies())
    df = pd.concat([df,dummies], axis = 1)
    df.drop([column],axis=1,inplace=True)

In [15]:
from sklearn import preprocessing
le_actors = preprocessing.LabelEncoder()

In [16]:
all_actors = pd.concat([df['actor_1_name'], df['actor_2_name'], df['actor_3_name']], ignore_index=True)


le_actors.fit(all_actors)
le_actors.classes_

numeric_actor_1 = le_actors.transform(df['actor_1_name'])
df['actor_1_name'] = numeric_actor_1


numeric_actor_2 = le_actors.transform(df['actor_2_name'])
df['actor_2_name'] = numeric_actor_2

numeric_actor_3 = le_actors.transform(df['actor_3_name'])
df['actor_3_name'] = numeric_actor_3

In [17]:
le_director_name = preprocessing.LabelEncoder()
le_director_name.fit(df['director_name'])
le_director_name.classes_

numeric_director = le_director_name.transform(df['director_name'])
df['director_name'] = numeric_director

In [18]:
df.drop(['movie_imdb_link', 'movie_title'], axis=1,inplace=True)

In [19]:
le_language = preprocessing.LabelEncoder()
le_language.fit(df['language'])
le_language.classes_

numeric_language = le_language.transform(df['language'])
df['language'] = numeric_language

In [20]:
le_country = preprocessing.LabelEncoder()
le_country.fit(df['country'])
le_country.classes_

numeric_country = le_country.transform(df['country'])
df['country'] = numeric_country

# SVR

SVR é um método de regressão supervisionado de machine learning. Ele é derivado do Support Vector Machine. Alguns elementos básicos do SVM ainda estão presentes como **maximun margin**. O SVR utiliza a função epsilon para ignorar os errors das obsevações que estão presentes dentro da **maximun margin**, também chamada de epsilon _intensive band_. Na execução do SVR as features são mapeadas para um espaço dimensional de _m_ dimensões usando um mapeamento não linear, então o modelo é construido utilizando esse espaço dimensional.


Em conjunto com a epsilon _intensive loss function_ é possível obter uma boa generalização dos dados
Referência: < http://kernelsvm.tripod.com/ >

In [21]:
df_test = df.drop(['plot_keywords'], axis=1)
X = df_test.drop(['imdb_score'],axis=1)
y = df_test['imdb_score']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

from  sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

from sklearn.linear_model import LinearRegression

from sklearn.svm import SVR



In [40]:
svr = SVR(kernel='linear')
svr.fit(X_train,y_train)
svr.score(X_test,y_test)

0.38692322832682696

In [29]:
svr = SVR(kernel='rbf')
svr.fit(X_train,y_train)
svr.score(X_test,y_test)

0.3434035464990782

In [30]:
from sklearn.svm import LinearSVR
svrl = LinearSVR()
svrl.fit(X_train,y_train)
svrl.score(X_test,y_test)

0.38419752616944447

# K-fold

## Linear Regression

In [16]:
from sklearn.model_selection import cross_val_score
cv_result = cross_val_score(LinearRegression(),X,y,cv=5)
print(np.mean(cv_result))

0.39817386175261066


## SVR Linear

In [22]:
cv_result = cross_val_score(SVR(kernel='linear'), X, y, cv=5)

In [23]:
np.mean(cv_result)

-12492514.265843943

## SVR rbf

In [24]:
cv_result = cross_val_score(SVR(kernel='rbf'), X, y, cv=5)

In [25]:
np.mean(cv_result)

0.006028421304931131